In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import copy

from autoencoder_linear import LayerNorm, FeedForward, clones, Encoder, Decoder

In [2]:
d_model, N, head_num, d_ff = 10, 3, 2, 20  # N: number of layer, head_num: number of head
encoder = Encoder(d_model, N, head_num, d_ff)
decoder = Decoder(d_model, N, head_num, d_ff)

In [3]:
batch_size, seq_len = 2, 64
x = torch.rand(batch_size, seq_len, d_model)
mask = torch.ones(batch_size, 1, seq_len)

In [4]:
memory = encoder(x, mask)

- encoder input: torch.Size([2, 64, 10])
- encoder: torch.Size([2, 64, 10])
- encoder: torch.Size([2, 64, 10])
- encoder: torch.Size([2, 64, 10])
- encoder: torch.Size([2, 32, 10])
- encoder: torch.Size([2, 16, 10])
- encoder: torch.Size([2, 8, 10])


In [5]:
memory.shape

torch.Size([2, 8, 10])

In [6]:
output = decoder(memory, torch.ones(batch_size, 1, memory.shape[1]))

- decoder input: torch.Size([2, 8, 10])
- decoder: torch.Size([2, 16, 10])
- decoder: torch.Size([2, 32, 10])
- decoder: torch.Size([2, 64, 10])
- decoder: torch.Size([2, 64, 10])
- decoder: torch.Size([2, 64, 10])
- decoder: torch.Size([2, 64, 10])


In [ ]:
output.shape

In [ ]:
log_softmax = nn.LogSoftmax(dim=1)
criterion = nn.NLLLoss()

In [ ]:
inputs = torch.tensor([[ 1.3956,  1.1497, -1.3392,  0.8979,  1.0989],
                       [-0.3986,  0.0929, -0.2454,  2.4595,  0.3489],
                       [ 0.4532,  0.6627,  0.6888, -0.9131, -1.8882]], requires_grad=True)
targets = torch.tensor([1, 0, 4])
print("Inputs: {}".format(inputs.shape))
print("Targets: {}".format(targets.shape))

In [ ]:
ls = torch.nn.functional.log_softmax(inputs, dim=-1)
print(ls)

In [ ]:
ls = criterion(ls, targets)

In [ ]:
ls.backward()

In [ ]:
inputs.grad

In [ ]:
# tensor([[ 0.1042, -0.2518,  0.0068,  0.0634,  0.0775],
#         [-0.3191,  0.0233,  0.0166,  0.2489,  0.0302],
#         [ 0.0866,  0.1068,  0.1096,  0.0221, -0.3250]])

In [ ]:
ls

In [ ]:
ls.mean()

In [ ]:
logits = torch.tensor([[[10, 1, 1],
                        [1, 10, 1],
                        [1, 1, 10]]], dtype=torch.float32)

In [ ]:
logits

In [ ]:
log_probs = F.log_softmax(logits, dim=-1)

In [ ]:
log_probs

In [ ]:
_, preds = torch.max(log_probs, dim=-1)

In [ ]:
preds

In [ ]:
labels = torch.tensor([[1, 2, 3, 4],
                       [2, 3, 4, 5]], dtype=torch.int32)
targets = torch.tensor([[1, 2, 3, 0],
                        [2, 3, 4, 0]], dtype=torch.int32)

In [ ]:
torch.sum(labels == targets).item() / torch.sum(torch.ones_like(labels)).item()

In [ ]:
labels == targets

In [ ]:
torch.mean((labels == targets).float()).item()

In [ ]:
a = torch.rand(1, 4, 2)

In [ ]:
a.shape

In [ ]:
a

In [ ]:
a.view(1, 2, 4)

In [ ]:
a

In [ ]:
# w_: [embedding_dim, num_embeddings]
# flat_inputs: [N, embedding_dim]

In [34]:
w_ = torch.tensor([[1., 2.],
                   [2., 3.],
                   [3., 4.]])
flat_inputs = torch.tensor([[2. , 3., 5.],
                            [1. , 2., 3.],
                            [2. , 4., 6.],
                            [2. , 4., 6.]])

In [35]:
(flat_inputs**2).sum(dim=1, keepdim=True)

tensor([[38.],
        [14.],
        [56.],
        [56.]])

In [36]:
(w_**2).sum(dim=0, keepdim=True)

tensor([[14., 29.]])

In [37]:
2 * torch.matmul(flat_inputs, w_)

tensor([[46., 66.],
        [28., 40.],
        [56., 80.],
        [56., 80.]])

In [38]:
distances = (flat_inputs**2).sum(dim=1, keepdim=True) - 2 * torch.matmul(flat_inputs, w_) + (w_**2).sum(dim=0, keepdim=True)

In [39]:
distances # [N, num_embeddings]

tensor([[ 6.,  1.],
        [ 0.,  3.],
        [14.,  5.],
        [14.,  5.]])

In [41]:
encoding_indices = torch.argmax(-distances, 1)

In [42]:
encoding_indices

tensor([1, 0, 1, 1])

In [47]:
one_hot = torch.zeros(encoding_indices.shape[0],
                                  2).scatter_(1, encoding_indices.unsqueeze(1), 1.)

In [48]:
one_hot

tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [0., 1.]])

In [56]:
torch.nn.functional.one_hot(encoding_indices, 2)

tensor([[0, 1],
        [1, 0],
        [0, 1],
        [0, 1]])

In [54]:
torch.mean(torch.nn.functional.one_hot(encoding_indices, 2), 0)

RuntimeError: Can only calculate the mean of floating types. Got Long instead.

In [8]:
distances = (tf.reduce_sum(flat_inputs**2, 1, keepdims=True)
             - 2 * tf.matmul(flat_inputs, w)
             + tf.reduce_sum(w ** 2, 0, keepdims=True))

NameError: name 'tf' is not defined

In [96]:
class VectorQuantizer(nn.Module):
    def __init__(self, dim_embedding, num_embeddings, commitment_cost):
        super(VectorQuantizer, self).__init__()
        self.dim_embedding = dim_embedding
        self.num_embeddings = num_embeddings
        self._commitment_cost = commitment_cost
        
        self.embed = nn.Embedding(num_embeddings, dim_embedding)
        
    def forward(self, inputs):
        """
        Args:
            inputs: [..., dim_embedding]
        """
        assert inputs.shape[-1] == self.dim_embedding
        flat_inputs = inputs.view(-1, inputs.shape[-1])
        
        # distance
        w = self.embed.weight.detach()
        with torch.no_grad():
            distances = ((flat_inputs**2).sum(dim=1, keepdim=True)
                         - 2 * torch.matmul(flat_inputs, w.T)  # distances: [N, num_embeddings]
                         + ((w.T)**2).sum(dim=0, keepdim=True))
            encoding_indices = torch.argmax(-distances, 1)  # [N]
            encodings = F.one_hot(encoding_indices, self.num_embeddings)
            encoding_indices = encoding_indices.view(inputs.shape[:-1])
        
        # get quantized vectors
        quantized = self.embed(encoding_indices)
        
        # compute loss
        q_latent_loss = torch.mean((inputs.detach() - quantized) ** 2)
        e_latent_loss = torch.mean((inputs - quantized.detach()) ** 2)
        loss = q_latent_loss + self._commitment_cost * e_latent_loss
        
        quantized = inputs + (quantized.detach() - inputs.detach())

        return {'quantize': quantized,
                'loss': loss,
                'encodings': encodings,
                'encoding_indices': encoding_indices,}

In [97]:
vq = VectorQuantizer(3, 10, 1)

In [105]:
inputs = torch.rand(2, 4, 3)

In [106]:
outputs = vq(inputs)

In [107]:
outputs['quantize'].shape

torch.Size([2, 4, 3])

In [102]:
inputs.shape

torch.Size([4, 3])